In [1]:
import numpy as np
import pandas as pd
import pickle

train = pd.read_csv('./raws/train_data.csv')
test = pd.read_csv('./raws/test_data.csv')

train = train.loc[:, 'lat':'MIN_WVP']
test = test.loc[:, 'lat':'MIN_WVP']

train = train.drop(columns=['Landsat_StartTime', 'PRODUCT_ID'])
test = test.drop(columns=['Landsat_StartTime', 'PRODUCT_ID'])

pickle.dump(train, open('./processed/shrinked_train.pkl', 'wb'))
pickle.dump(test, open('./processed/shrinked_test.pkl', 'wb'))

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

# 以下本体

In [57]:
def featurecreation(df, istrain:bool):
    # 切り出し
    # df = df.loc[:, 'lat':'sst_diff']
    df = df.loc[:, 'lat':'MIN_WVP']

    # 周辺調査地のデータ挿入
    a = list(df.loc[:, 'lat'])
    o = list(df.loc[:, 'lon'])
    group = [str(round(a[i], 2))+'_'+str(round(o[i], 2)) for i in range(len(a))]
    # group = [str(round(a[i], 1))+'_'+str(round(o[i], 1)) for i in range(len(a))]
    if istrain == True:
        global c, temp
        c = list(df.loc[:, 'cover'])
        temp = {i: [[a[j], o[j], c[j]] for j in range(len(c)) if group[j]==i]
                for i in np.unique(group)}
    
    # dist = [{(a[j]-i[0])**2 + (o[j]-i[1])**2 : i[2] for i in temp[group[j]]}
    #         for j in range(len(a))]
    # dist = [sorted(i.items()) + [(100, np.nan)]*3 for i in dist]

    dist = []
    for i in range(len(a)):
        td = {}
        try:
            for j in temp[group[i]]:
                td[(a[i]-j[0])**2 + (o[i]-j[1])**2] = j[2]
        except:
            td = {1: np.nan, 2: np.nan, 3: np.nan}
        dist.append(td)

    dist = [sorted(i.items()) for i in dist]
    n1 = []
    n3 = []
    for i in dist:
        # cover_nearest
        try:
            if istrain == True:
                n1.append(i[1][1])
            else:
                n1.append(i[0][1])
        except (KeyError, IndexError):
            n1.append(np.nan)
        # cover_3
        try:
            if istrain == True:
                n3.append(mean([i[1][1], i[2][1], i[3][1]]))
            else:
                n3.append(mean([i[0][1], i[1][1], i[2][1]]))
        except (KeyError, IndexError):
            n3.append(np.nan)

    df['cover_nearest'] = n1
    df['cover_3'] = n3
    
    # dd_df = df.loc[:, ['lon', 'lat']].values
    # if istrain == True:
    #     nbrs = NearestNeighbors(n_neighbors=3).fir(dd_df)
    
    # # 海岸の比率
    # df['beach_rate'] = df['beach_length'] / df['coast_length']
    # df['aicial_rate'] = df['aicial_length'] / df['coast_length']

    # 透明度推定
    # df['SS_est'] = (df['Green']-df['SWIR2']) / (df['Red']-df['SWIR2'])
    df['Chla_est'] = (df['Blue']-df['SWIR2']) / (df['Red']-df['SWIR2'])

    # その他
    df['slope'] = df['depth_original'] / df['coastal_dist']
    # df['sdiff'] = df['warm_sst'] - df['cold_sst']

    # depthの処理
    # df.loc[df['depth']==0, 'depth'] = np.nan
    # df['dfdiff'] = df['depth_original'] - df['depth']
    # df.loc[~(df['depth_original'].isna()), 'depth'] = np.nan

    # 文献の分類
    # df['YMDstyle'] = [str(str(i).count('-')) +'_'+ str(str(i).count('.')) for i in list(df['YMD'])]
    # df['YMDstyle'].replace(['0_4', '1_2', '2_12', '0_2'], np.nan, inplace=True)
    # df['YMDstyle'].replace(['0_0', '0_1', '0_2', '0_4', '1_2', '1_4', '2_12'], [2, 1, 3, np.nan, np.nan, 4, np.nan], inplace=True)

    # 必要な列の抽出
    col = ['Chla_est', 'cover_nearest', 'slope', 'cover_3']
    # col = ['Chla_est', 'cover_3']
    df = pd.concat([df.loc[:, 'lat':'sst_diff'], df.loc[:, col]], axis=1)
    col = ['year', 'month', 'YMD', 'area', 'lon', 'lat']
    col = col + ['cliff_length', 'Salinity_annual', 'river_area', 'sst_diff']
    col = col + ['cold_sst', 'coast_length', 'sst_ymd', 'sst_annual']
    # col = col + ['river_dist', 'aicial_length', 'beach_length', 'coastal_dist', 'depth']
    df.drop(columns=col, inplace=True)
    # df = df.loc[:, ['cover', 'depth_original','area','YMDstyle','warm_sst','fetch','depth','hist_warm_sst','coastal_dist','Salinity_annual','coast_length','aicial_length','sst_diff']]

    # スタッキング用にバリデーションのFoldを追加
    df_ = pd.DataFrame()
    if istrain == True:
        l = [i for i in range(4) for j in range(math.floor(len(df)/4))] + [3]*(len(df)-(round(len(df)/4)*4))
        df['valfold'] = random.sample(l, len(l))
        df_ = df
        # for i in range(4):
        #     df_ = pd.concat([df_, df.replace({'valfold': [(i+j+1)%4 for j in range(3)]}, i*11)])
    elif istrain == False:
        df_ = df    

    return df_

In [50]:
def na_estimation(df, estlist):
    model_dict = {}

    df = df.iloc[:14139, :]

    for col in estlist:
        print('\n===========================================================\n'
        'estimating ' + col + '\n')

        train_df = df.loc[~(df[col].isna()), ]

        x = train_df.drop(columns=['cover', 'valfold', col], errors='ignore')
        y = train_df[col]

        kf = KFold(n_splits=4, shuffle=True, random_state=234)
        c = 0
        model = []
        
        for idx_tr, idx_va in kf.split(train_df):
            tr_x, va_x = x.iloc[idx_tr], x.iloc[idx_va]
            tr_y, va_y = y.iloc[idx_tr], y.iloc[idx_va]

            lgb_train = lgb.Dataset(tr_x, tr_y)
            lgb_val = lgb.Dataset(va_x, va_y)

            params = {
                'objective': 'regression',
                'metric': 'rmse',
                'verbose': -1,
                'random_state': 254,
                # 'min_data_in_leaf': 200,
                # 'bagging_fraction': 0.5,
                # 'bagging_freq': 1,
                'lambda_l1': 0.5,
                'lambda_l2': 0.5,
            }

            model.append(lgb.train(
                params,
                lgb_train,
                valid_sets=[lgb_train, lgb_val],
                valid_names=['train', 'val'],
                num_boost_round=1000,
                callbacks=[lgb.early_stopping(stopping_rounds=2, verbose=True)]
            ))
        model_dict[col] = model

    print('\n===========================================================\n'
    'finished estimating.')
    
    return model_dict

def predict_na(df, model):
    df_ = df

    for col in model.keys():
        for i in range(4):
            df_[col + 'pred' + str(i)] = model[col][i].predict(df, predict_disable_shape_check=True)
        
        df_[col + '_pred'] = df_[[col+'pred'+str(i) for i in range(4)]].mean(axis=1)
        df_[col].fillna(df_[col+'_pred'], inplace=True)
        df_.drop(columns=[col+'pred'+str(i) for i in range(4)]+[col+'_pred'], inplace=True)

    return df_

def make_nn_data(df, istrain:bool):
    # NA推定
    estlist = ['depth_original', 'Chla_est']
    if istrain == True:
        global fillmodel
        fillmodel = na_estimation(df, estlist)
    df = predict_na(df, fillmodel)

    # 穴埋め
    for col in df.columns:
        df[col].fillna(df[col].mean(numeric_only=True), inplace=True)
    
    #標準化
    if istrain == True:
        global mea
        global std
        mea = df.drop(columns=['cover', 'valfold']).mean()
        std = df.drop(columns=['cover', 'valfold']).std()
        cover = df['cover']
        valfold = df['valfold']
        df = (df - mea) / std
        df['cover'] = cover
        df['valfold'] = valfold
    else:
        df = (df - mea) / std

    return df

In [51]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import pickle
from sklearn.model_selection import KFold
import random
from sklearn.neighbors import NearestNeighbors
from statistics import mean
import math

# train = pd.read_csv('./raws/train_data.csv')
# test = pd.read_csv('./raws/test_data.csv')
train = pickle.load(open('./processed/shrinked_train.pkl', 'rb'))
test = pickle.load(open('./processed/shrinked_test.pkl', 'rb'))

In [58]:
train_lgb = featurecreation(train, True)
test_lgb = featurecreation(test, False)

# df = train_lgb[train_lgb['cover']>0.5]
# for i in range(5):
#     train_lgb = pd.concat([train_lgb, df])
# train_lgb.reset_index(drop=True, inplace=True)

pickle.dump(train_lgb, open('./processed/nan_train.pkl', 'wb'))
pickle.dump(test_lgb, open('./processed/nan_test.pkl', 'wb'))

train_nn = make_nn_data(train_lgb, True)
test_nn = make_nn_data(test_lgb, False)

pickle.dump(train_nn, open('./processed/filled_train.pkl', 'wb'))
pickle.dump(test_nn, open('./processed/filled_test.pkl', 'wb'))


estimating depth_original

Training until validation scores don't improve for 2 rounds
Early stopping, best iteration is:
[211]	train's rmse: 0.262824	val's rmse: 0.334566
Training until validation scores don't improve for 2 rounds
Early stopping, best iteration is:
[55]	train's rmse: 0.322443	val's rmse: 0.616311
Training until validation scores don't improve for 2 rounds
Early stopping, best iteration is:
[88]	train's rmse: 0.293195	val's rmse: 0.463553
Training until validation scores don't improve for 2 rounds
Early stopping, best iteration is:
[86]	train's rmse: 0.354123	val's rmse: 0.402986

estimating Chla_est

Training until validation scores don't improve for 2 rounds
Early stopping, best iteration is:
[61]	train's rmse: 0.230031	val's rmse: 0.319952
Training until validation scores don't improve for 2 rounds
Early stopping, best iteration is:
[96]	train's rmse: 0.204161	val's rmse: 0.290772
Training until validation scores don't improve for 2 rounds
Early stopping, best iter

In [61]:
train_lgb

,cover,depth_original,aicial_length,beach_length,coastal_dist,depth,fetch,hist_cold_sst,hist_warm_sst,river_dist,warm_sst,Chla_est,cover_nearest,slope,cover_3,valfold
0,0.050,6.218322,0.000000,0.000000,5590.0,23.664240,2590.515869,20.798887,31.061762,126.328598,29.149746,3.648817,0.050,-inf,0.050,2
6,0.075,7.242031,0.000000,877.570068,200.0,24.079815,1153.718506,17.963797,31.112072,7.336063,29.178904,3.353233,0.075,-inf,0.075,0
7,0.150,2.000000,0.000000,0.000000,4100.0,1.000000,1718.315796,20.647753,31.402370,10.295314,29.357115,3.926495,0.150,0.000488,0.150,2
9,0.050,8.105741,0.000000,1329.338867,117.5,5.412169,1144.260986,20.089460,31.146654,37.656109,28.946106,4.019800,0.050,-inf,0.050,1
11,0.000,0.400000,0.000000,579.048828,338.0,6.978470,508.141266,21.131441,31.394224,3.806094,29.387054,2.949698,0.000,0.001183,0.000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14124,0.025,8.554358,14.368234,855.136963,325.0,9.134441,661.605347,17.726496,31.248995,5.513156,29.086094,3.085735,0.025,-inf,0.025,0
14130,0.050,6.218322,0.000000,0.000000,8300.0,21.561544,2617.064453,20.805063,31.091637,127.528763,29.172035,3.928402,0.050,-inf,0.050,3
14131,0.375,0.900000,0.000000,1175.292236,330.0,0.879137,1127.085205,21.265642,31.285402,15.682171,29.331314,3.540347,0.375,0.002727,0.375,0
14135,0.050,8.105741,0.000000,56.749870,10.0,0.322502,1372.131348,20.803297,31.215786,128.361176,29.155453,3.385380,0.050,-inf,0.050,1
